## Objective
Parses a text file table that has reinsurance and risk adjustment data for a benefit year into a CSV.

### Preprocessing
step 1: copy into raw text file

step 2: delete header 

HIOS ID
HIOS INPUTTED
INSURANCE COMPANY
NAME STATE
REINSURANCE
PAYMENT
AMOUNT (OR
NOT ELIGIBLE)
HHS RISK
ADJUSTMENT
TRANSFER
AMOUNT
(INDIVIDUAL
MARKET,
INCLUDING
CATASTROPHIC)
HHS RISK
ADJUSTMENT
TRANSFERS
AMOUNT (SMALL
GROUP MARKET)

step 3: delete page numbers through the regexp ^RI RA.\*

step 4: remove blank lines through the regexp ^\n\n

step 5: find and replace "not eligible" with "N/E"

step 6: find and replace "\$ -" with "\$-"

step 7: find and replace "\$ " with "\$" (also works for step 6)

step 8: find and replace "N/A - MA Issuer" with "N/A_MA_Issuer" (dataframe has trouble parsing "-")

step 9: find and replace "N/A - Default Charge" with "N/A_DefaultCharge"

step 10: delete footnotes on the first page

In [105]:
import pandas as pd
# import re

In [106]:
#with open('2014-Benefit-Year-Edit.txt', 'r') as myfile:
with open('input/2016-Benefit-Year-Edit.txt', 'r') as myfile:
    s = myfile.read()

In [107]:
split = str.split(s)
length = len(split)

In [108]:
storage = pd.DataFrame(columns=["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME", "STATE", "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)",
               "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
               "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)"])

# prog = re.compile('^[A-Z]{2}$')

i = 0
index = 0
while i < length - 1:
    hios_id = split[i]
    i = i + 1
    temp = i
    # while (prog.match(split[i]) == None or split[i] in ['US', 'BC', 'OF']): # look for state in quotes
    while (split[i] != 'N/E' and split[i][:1] != '$'):
        i = i + 1
    company_name = str(' '.join(split[temp:i - 1]))
    state = split[i - 1]
    reinsurance = split[i]
    i = i + 1
    individual = split[i]
    i = i + 1
    small_group = split[i] 
    i = i + 1
    storage.loc[index] = [hios_id, company_name, state, reinsurance, individual, small_group]
    index = index + 1
    
print(storage)

    HIOS ID               HIOS INPUTTED INSURANCE COMPANY NAME STATE  \
0     11082                       Aetna Life Insurance Company    AK   
1     38344                                 Premera Blue Cross    AK   
2     73836                               Moda Health Plan Inc    AK   
3     80049                 UnitedHealthcare Insurance Company    AK   
4     44580                           Humana Insurance Company    AL   
5     44902                 Federated Mutual Insurance Company    AL   
6     46944              Blue Cross and Blue Shield of Alabama    AL   
7     59809            UnitedHealthcare Life Insurance Company    AL   
8     68259                  UnitedHealthcare of Alabama, Inc.    AL   
9     69461                 UnitedHealthcare Insurance Company    AL   
10    93018                                  Viva Health, Inc.    AL   
11    93122          Freedom Life Insurance Company of America    AL   
12    13262                    USAble Mutual Insurance Company  

In [109]:
# save it
storage.to_csv("output/2016-Benefit-Year-RI-RA.csv")